<center><img src="images/header.png"></center>

<h1><center>Алгоритмы интеллектуальной обработки больших объемов данных</center></h1>
<hr>
<h2><center>Введение в рекомендательные системы</center></h2>

In [2]:
%matplotlib inline

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)

# Подготовка данных

1. Скачайте [данные](https://cloud.mail.ru/public/CSjR/mPctRVc2u) о рейтингах и фильмах
2. Загрузите рейтинги (user_ratedmovies.dat) и описание фильмов (movies.dat)
3. С помощью LabelEncoder перекодируйте идентификаторы фильмов и юзеров в обоих датафреймах


In [8]:
df_movies = pd.read_csv('./data/movies.dat', sep='\t', encoding='latin1')

In [9]:
df_ratings = pd.read_csv('./data/user_ratedmovies.dat', sep='\t', encoding='latin1')

In [11]:
df_ratings.head()

,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second
0,75,3,1.0,29,10,2006,23,17,16
1,75,32,4.5,29,10,2006,23,23,44
2,75,110,4.0,29,10,2006,23,30,8
3,75,160,2.0,29,10,2006,23,16,52
4,75,163,4.0,29,10,2006,23,29,30


In [10]:
df_movies.head()

,id,title,imdbID,spanishTitle,imdbPictureURL,year,rtID,rtAllCriticsRating,rtAllCriticsNumReviews,rtAllCriticsNumFresh,...,rtAllCriticsScore,rtTopCriticsRating,rtTopCriticsNumReviews,rtTopCriticsNumFresh,rtTopCriticsNumRotten,rtTopCriticsScore,rtAudienceRating,rtAudienceNumRatings,rtAudienceScore,rtPictureURL
0,1,Toy story,114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995,toy_story,9,73,73,...,100,8.5,17,17,0,100,3.7,102338,81,http://content7.flixster.com/movie/10/93/63/10...
1,2,Jumanji,113497,Jumanji,http://ia.media-imdb.com/images/M/MV5BMzM5NjE1...,1995,1068044-jumanji,5.6,28,13,...,46,5.8,5,2,3,40,3.2,44587,61,http://content8.flixster.com/movie/56/79/73/56...
2,3,Grumpy Old Men,107050,Dos viejos gruñones,http://ia.media-imdb.com/images/M/MV5BMTI5MTgy...,1993,grumpy_old_men,5.9,36,24,...,66,7,6,5,1,83,3.2,10489,66,http://content6.flixster.com/movie/25/60/25602...
3,4,Waiting to Exhale,114885,Esperando un respiro,http://ia.media-imdb.com/images/M/MV5BMTczMTMy...,1995,waiting_to_exhale,5.6,25,14,...,56,5.5,11,5,6,45,3.3,5666,79,http://content9.flixster.com/movie/10/94/17/10...
4,5,Father of the Bride Part II,113041,Vuelve el padre de la novia (Ahora también abu...,http://ia.media-imdb.com/images/M/MV5BMTg1NDc2...,1995,father_of_the_bride_part_ii,5.3,19,9,...,47,5.4,5,1,4,20,3,13761,64,http://content8.flixster.com/movie/25/54/25542...


In [12]:
df_ratings.userID.max()

71534

In [14]:
df_ratings.movieID.max()

65133

In [15]:
df_ratings.movieID.nunique()

10109

In [13]:
df_ratings.userID.nunique()

2113

In [16]:
from sklearn.preprocessing import LabelEncoder

In [17]:
enc_user = LabelEncoder()
enc_movie = LabelEncoder()

In [19]:
enc_user = enc_user.fit(df_ratings.userID.values)

In [20]:
enc_movie = enc_movie.fit(
    np.r_[df_ratings.movieID.values, df_movies.loc[:, 'id'].values]
)

In [21]:
df_ratings.loc[:, 'userID'] = \
    enc_user.transform(df_ratings.loc[:, 'userID'])
df_ratings.loc[:, 'movieID'] = \
    enc_movie.transform(df_ratings.loc[:, 'movieID'])

In [22]:
df_movies.loc[:, 'id'] = enc_movie.transform(df_movies.loc[:, 'id'])

# Сжатое представление фильмов

1. С помощью from scipy.sparse.coo_matrix составьте разреженную матрицу рейтингов
2. С помощью scipy.sparse.linalg.svds получите латентное описание фильмов и пользователей
3. Для каждого фильма найдите 10 ближайших соседей в этих признаках по косинусной мере

In [25]:
from scipy.sparse import coo_matrix

In [26]:
ratings = df_ratings.rating.values
user_idx = df_ratings.userID.values
item_idx = df_ratings.movieID.values

In [29]:
R = coo_matrix((ratings, (user_idx, item_idx)))

In [31]:
from scipy.sparse.linalg import svds

In [32]:
u, s, vt = svds(R, k=10)

In [33]:
u.shape

(2113, 10)

In [34]:
vt.shape

(10, 10197)

In [35]:
from sklearn.neighbors import NearestNeighbors

In [36]:
# На выходе - матрица с индексами ближайших соседей для каждого объекта
nn = NearestNeighbors()
V = vt.T
nn.fit(V)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=5, p=2, radius=1.0)

In [37]:
res = nn.kneighbors(V, n_neighbors=10, )

In [43]:
df_movies = df_movies.sort_values('id')

In [44]:
movie_titles = df_movies.title.values

In [50]:
res[1]

array([[    0,   565,  2873, ...,   351,  4007,  2404],
       [    1,  1848,   149, ...,  5178,   144,   692],
       [    2,     4,   414, ...,  1326,   934,   505],
       ...,
       [10194,   780,  5210, ..., 10045,  3883,  9569],
       [10166, 10195,  5079, ...,  9850,  9567,  7535],
       [10196,  2696,  7495, ...,  4002,  3266,  8399]])

In [52]:
movie_nn_names = pd.DataFrame(data=movie_titles[res[1]], 
                              columns=['movie']+['nn_{}'.format(i) for i in range(9)])

In [65]:
idx = movie_nn_names.movie.str.contains(u'Starship', case=False)
movie_nn_names.loc[idx]

,movie,nn_0,nn_1,nn_2,nn_3,nn_4,nn_5,nn_6,nn_7,nn_8
1509,Starship Troopers,Alien³,Pitch Black,Conan the Barbarian,Blade,Johnny Mnemonic,Demolition Man,Judge Dredd,The X Files,Event Horizon
10160,Starship Troopers 3: Marauder,Impulse,Bedtime Stories,Anamorph,Cobra,Silk,Rogue,Flaming Star,The Reader,Revolutionary Road


# User-based CF

* Разбейте данные на обучение и контроль в пропорции 80/20
* Реализуйте функцию расчета попарных схожестей между пользователями
* Реализуйте метод расчета рекомендаций на основе $K$ наиболее похожих пользователей. Постройте график зафисимости ошибки MAE от $K$ (5-25)
* Выполните нормализацию рейтингов с помощью вычитания средней оценки $\bar{R}_u$ каждого из пользователей и повторите предыдущий 2 шага. В этом случае предсказание выполняется следующим образом
$$ \hat{R}_{ui} = \bar{R}_u + \frac{\sum_{v \in N(u)} s_{uv} (R_{vi} - \bar{R}_v)}{\sum_{v \in N(u)}| s_{uv}|} $$
* Перейдите к Item-Based подходу и повторите предыдущие шаги

In [67]:
from scipy.spatial.distance import cosine
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
from scipy.spatial.distance import correlation
from sklearn.metrics import pairwise_distances

In [ ]:
def my_similarity(u, v):
    # Функция возвращает меру похожести между
    # рейтингами пользователей u и v
    
    return similarity

In [ ]:
pd.to_datetime()

In [66]:
df_ratings.head()

,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second
0,0,2,1.0,29,10,2006,23,17,16
1,0,31,4.5,29,10,2006,23,23,44
2,0,105,4.0,29,10,2006,23,30,8
3,0,151,2.0,29,10,2006,23,16,52
4,0,154,4.0,29,10,2006,23,29,30
